In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

# Directory where your images are located
dataset_dir = r"D:\SEM-4\PROJECTS\ML\Archive"

# Initialize VGG16 model, this time including the top layers
base_model = VGG16(include_top=True, weights='imagenet')

# However, instead of using the model as is, create a new model that outputs the features from the penultimate layer
# The penultimate layer is the one before the final classification layer, and it has 4096 features
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

# Create an instance of the ImageDataGenerator for loading images
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Create a data generator for reading images from directories
generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,  # Adjust based on your GPU memory
    class_mode='sparse',  # 'sparse' yields integer labels
    shuffle=False  # Important for keeping labels in order
)

# Number of images and labels
num_images = generator.samples
print("Number of images found:", num_images)
num_classes = generator.num_classes
print("Number of classes found:", num_classes)

# Extract features
features = model.predict(generator, steps=np.ceil(num_images/32), verbose=1)

# Get the labels (ensure they are in the same order as the images)
labels = generator.classes

# Saving features and labels to .npy files
np.save('features_4096.npy', features)
np.save('labels_4096.npy', labels)





Found 42500 images belonging to 13 classes.
Number of images found: 42500
Number of classes found: 13
   2/1329 [..............................] - ETA: 39:19

C:\Users\garik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


  24/1329 [..............................] - ETA: 48:20

C:\Users\garik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\PIL\Image.py:3182: DecompressionBombWarning: Image size (96714256 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


1110/1329 [========================>.....] - ETA: 6:18

C:\Users\garik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\PIL\TiffImagePlugin.py:868: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


1329/1329 [==============================] - 2277s 2s/step


In [3]:
import numpy as np
features = np.load(r'D:\SEM-4\ML\CODES\Machine-Learning\features_4096.npy')
size = np.shape(features)
print(size)

labels = np.load(r'D:\SEM-4\ML\CODES\Machine-Learning\labels_4096.npy')
size1 = np.shape(labels)
print(size1)

(42500, 4096)
(42500,)


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Assuming features and labels are already loaded from the .npy files
features = np.load('D://SEM-4//ML//CODES\Machine-Learning//features_4096.npy')
labels = np.load('D:\SEM-4\ML\CODES\Machine-Learning\labels_4096.npy')

# Reshape features for kNN
features = features.reshape(features.shape[0], -1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define the parameter grid: number of neighbors
param_grid = {'n_neighbors': [3, 5, 7, 9, 11, 13, 15]}

# Initialize a kNN classifier
knn = KNeighborsClassifier()

# Initialize the GridSearchCV object
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')

# Fit it to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy found: ", grid_search.best_score_)

# Evaluate on the test set with the best parameters
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))


Best parameters found:  {'n_neighbors': 15}
Best accuracy found:  0.5213823529411765
              precision    recall  f1-score   support

           0       0.56      0.38      0.45       276
           1       0.77      0.46      0.58       646
           2       0.39      0.66      0.49      1081
           3       0.63      0.45      0.53       506
           4       0.69      0.63      0.66       448
           5       0.60      0.61      0.60       594
           6       0.94      0.31      0.47       269
           7       0.48      0.61      0.54      1088
           8       0.60      0.57      0.58      1204
           9       0.33      0.54      0.41       498
          10       0.61      0.49      0.55      1352
          11       0.42      0.12      0.18       318
          12       0.88      0.65      0.75       220

    accuracy                           0.53      8500
   macro avg       0.61      0.50      0.52      8500
weighted avg       0.57      0.53      0.53      

In [7]:
import numpy as np

# Load your labels if not already loaded
labels = np.load('D:\SEM-4\ML\CODES\Machine-Learning\labels_4096.npy')
unique, counts = np.unique(labels, return_counts=True)
class_distribution = dict(zip(unique, counts))

# Printing each class with its count on a new line
for class_label, count in class_distribution.items():
    print(f"Class {class_label}: {count}")


Class 0: 1305
Class 1: 3035
Class 2: 5312
Class 3: 2607
Class 4: 2235
Class 5: 3115
Class 6: 1324
Class 7: 5373
Class 8: 6192
Class 9: 2521
Class 10: 6813
Class 11: 1510
Class 12: 1158


Before PCA:

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Paths to the features and labels files
features_path = 'D://SEM-4//ML//CODES//Machine-Learning//features_4096.npy'
labels_path = 'D:\SEM-4\ML\CODES\Machine-Learning\labels_4096.npy'

# Load features and labels
features = np.load(features_path)
labels = np.load(labels_path)

# Reshape features from 4D (n_samples, height, width, channels) to 2D (n_samples, height*width*channels)
features = features.reshape(features.shape[0], -1)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Dictionary of classifiers
classifiers = {
    "CatBoost": CatBoostClassifier(verbose=0),
    "XGBoost": XGBClassifier(),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB()
}

# Results dictionary
results = {}

for name, clf in classifiers.items():
    # Train the classifier
    clf.fit(X_train, y_train)
    # Predict the responses for the test dataset
    y_pred = clf.predict(X_test)
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred)
    # Store results
    results[name] = (accuracy, precision, recall, f1, conf_matrix)

# Print all results
for name, metrics in results.items():
    print(f"{name} Performance Metrics:")
    print(f"Accuracy: {metrics[0]:.2f}")
    print(f"Precision: {metrics[1]:.2f}")
    print(f"Recall: {metrics[2]:.2f}")
    print(f"F1 Score: {metrics[3]:.2f}")
    print("Confusion Matrix:")
    print(metrics[4])
    print("\n")

CatBoost Performance Metrics:
Accuracy: 0.59
Precision: 0.59
Recall: 0.59
F1 Score: 0.58
Confusion Matrix:
[[ 158    6   36    5    8   16    0   38   51   20   64    2    4]
 [   6  587   11   51   63   17   14   36   66    5   61    9   10]
 [  10   14  942   13    2   28    9  106  244   73  180    7    0]
 [   1   46   24  442   30   14   19   49   60    2   47   17    9]
 [   2   58    2   24  497    5    8   21   26    2   23    5    2]
 [  17   28   85    5    3  516    4   14   84   52   89    3    1]
 [   0   23   12   32   11    6  215   22   39    0   21    6    3]
 [   9   16  112   28   12   20    6 1009   79   29  265    7    4]
 [   6   39  231   35   14   36    9   58 1306   26   61   11   30]
 [  16    8  148    5    2   28    1   39   74  302  113    1    1]
 [  24   36  150   26    4   54   10  247  179   69 1220   18    8]
 [   2   19   36   42   16   14    9  112   86    3   58   73    7]
 [   0   13    2   12    1    0    2    1   54    0    6    2  241]]


XGBoos

PCA to explain 99% variance:

In [3]:
import numpy as np
from sklearn.decomposition import PCA

# Paths to the features and labels files
features_path = 'D://SEM-4//ML//CODES//Machine-Learning//features_4096.npy'
labels_path = 'D:\SEM-4\ML\CODES\Machine-Learning\labels_4096.npy'# Load features and labels
features = np.load(features_path)
labels = np.load(labels_path)

# Reshape features from 4D (n_samples, height, width, channels) to 2D (n_samples, height*width*channels)
features = features.reshape(features.shape[0], -1)

# Applying PCA to capture 99% of the variance
pca = PCA(0.99)
features_pca = pca.fit_transform(features)

# Saving the reduced features to a new file
reduced_features_path = 'D://SEM-4//ML//CODES//Machine-Learning//Reduced//reduced_features.npy'
np.save(reduced_features_path, features_pca)

# Optionally, save the labels if you need to keep them aligned with the reduced features for later use
reduced_labels_path = 'D://SEM-4//ML//CODES//Machine-Learning//Reduced//reduced_labels.npy'
np.save(reduced_labels_path, labels)

# Number of components selected
n_components = pca.n_components_
print(f"Number of principal components selected to explain at least 99% of the variance: {n_components}")

Number of principal components selected to explain at least 99% of the variance: 3300


After PCA:

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Paths to the features and labels files
features_path = 'D://SEM-4//ML//CODES//Machine-Learning//Reduced//reduced_features.npy'
labels_path = 'D://SEM-4//ML//CODES//Machine-Learning//Reduced//reduced_labels.npy'

# Load features and labels
features = np.load(features_path)
labels = np.load(labels_path)

# Reshape features from 4D (n_samples, height, width, channels) to 2D (n_samples, height*width*channels)
features = features.reshape(features.shape[0], -1)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Dictionary of classifiers
classifiers = {
    "CatBoost": CatBoostClassifier(verbose=0),
    "XGBoost": XGBClassifier(),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB()
}

# Results dictionary
results = {}

for name, clf in classifiers.items():
    # Train the classifier
    clf.fit(X_train, y_train)
    # Predict the responses for the test dataset
    y_pred = clf.predict(X_test)
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred)
    # Store results
    results[name] = (accuracy, precision, recall, f1, conf_matrix)

# Print all results
for name, metrics in results.items():
    print(f"{name} Performance Metrics:")
    print(f"Accuracy: {metrics[0]:.2f}")
    print(f"Precision: {metrics[1]:.2f}")
    print(f"Recall: {metrics[2]:.2f}")
    print(f"F1 Score: {metrics[3]:.2f}")
    print("Confusion Matrix:")
    print(metrics[4])
    print("\n")